In [1]:
import math
import torch
import gpytorch
import tqdm
from matplotlib import pyplot as plt
import numpy as np
import os
import random

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def load_dataset(dataset_dir):
    data = {}

    for category in ['train_l1', 'train_l2', 'val_l1', 'val_l2', 'test_l1', 'test_l2']:
        cat_data = np.load(os.path.join(dataset_dir, category + '.npz'))
        data['x_' + category] = cat_data['x']
        data['y_' + category] = cat_data['y']
    
    return data

data = load_dataset('nargp_data')

x_train_l = data['x_train_l1']
x_train_l = x_train_l.reshape(x_train_l.shape[0],-1)
y_train_l = data['y_train_l1']
y_train_l = y_train_l.reshape(y_train_l.shape[0],-1)
x_train_h = data['x_train_l2']
x_train_h = x_train_h.reshape(x_train_h.shape[0],-1)
y_train_h = data['y_train_l2']
y_train_h = y_train_h.reshape(y_train_h.shape[0],-1)

x_val_l = data['x_val_l1']
x_val_l = x_val_l.reshape(x_val_l.shape[0],-1)
y_val_l = data['y_val_l1']
y_val_l = y_val_l.reshape(y_val_l.shape[0],-1)
x_val_h = data['x_val_l2']
x_val_h = x_val_h.reshape(x_val_h.shape[0],-1)
y_val_h = data['y_val_l2']
y_val_h = y_val_h.reshape(y_val_h.shape[0],-1)

x_test_l = data['x_test_l1']
x_test_l = x_test_l.reshape(x_test_l.shape[0],-1)
y_test_l = data['y_test_l1']
y_test_l = y_test_l.reshape(y_test_l.shape[0],-1)
x_test_h = data['x_test_l2']
x_test_h = x_test_h.reshape(x_test_h.shape[0],-1)
y_test_h = data['y_test_l2']
y_test_h = y_test_h.reshape(y_test_h.shape[0],-1)

train_x_l = torch.from_numpy(x_train_l)
train_y_l = torch.from_numpy(y_train_l)

train_x_h = torch.from_numpy(x_train_h)
train_y_h = torch.from_numpy(y_train_h)

val_x_l = torch.from_numpy(x_val_l)
val_y_l = torch.from_numpy(y_val_l)

val_x_h = torch.from_numpy(x_val_h)
val_y_h = torch.from_numpy(y_val_h)

test_x_l = torch.from_numpy(x_test_l)
test_y_l = torch.from_numpy(y_test_l)

test_x_h = torch.from_numpy(x_test_h)
test_y_h = torch.from_numpy(y_test_h)

print(y_train_l.shape,y_train_h.shape,y_val_l.shape,y_val_h.shape,y_test_l.shape,y_test_h.shape)

(87, 1176) (32, 45414) (50, 1176) (50, 45414) (50, 1176) (50, 45414)


In [3]:
num_latents = 1
num_tasks_l1 = 1176

class MultitaskGPModel_l1(gpytorch.models.ApproximateGP):
    def __init__(self):
        # Let's use a different set of inducing points for each latent function
        inducing_points = torch.rand(num_latents, 1, 1176)

        # We have to mark the CholeskyVariationalDistribution as batch
        # so that we learn a variational distribution for each task
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(-2), batch_shape=torch.Size([num_latents])
        )

        # We have to wrap the VariationalStrategy in a LMCVariationalStrategy
        # so that the output will be a MultitaskMultivariateNormal rather than a batch output
        variational_strategy = gpytorch.variational.LMCVariationalStrategy(
            gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True
            ),
            num_tasks=1176,
            num_latents=1,
            latent_dim=-1
        )

        super().__init__(variational_strategy)

        # The mean and covariance modules should be marked as batch
        # so we learn a different set of hyperparameters
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([num_latents]))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([num_latents])),
            batch_shape=torch.Size([num_latents])
        )

    def forward(self, x):
        # The forward function should be written as if we were dealing with each output
        # dimension in batch
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [4]:
num_latents = 1
num_tasks_l2 = 45414 #45414+1176

class MultitaskGPModel_l2(gpytorch.models.ApproximateGP):
    def __init__(self):
        # Let's use a different set of inducing points for each latent function
        inducing_points = torch.rand(num_latents, 1, 46590)

        # We have to mark the CholeskyVariationalDistribution as batch
        # so that we learn a variational distribution for each task
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(-2), batch_shape=torch.Size([num_latents])
        )

        # We have to wrap the VariationalStrategy in a LMCVariationalStrategy
        # so that the output will be a MultitaskMultivariateNormal rather than a batch output
        variational_strategy = gpytorch.variational.LMCVariationalStrategy(
            gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True
            ),
            num_tasks= 45414,
            num_latents=1,
            latent_dim=-1
        )

        super().__init__(variational_strategy)

        # The mean and covariance modules should be marked as batch
        # so we learn a different set of hyperparameters
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([num_latents]))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([num_latents]),active_dims=tuple(range(0,1176)))*
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([num_latents]),active_dims=tuple(range(1176,46590)))+
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([num_latents]),active_dims=tuple(range(1176,46590))),
            batch_shape=torch.Size([num_latents])
        )

    def forward(self, x):
        # The forward function should be written as if we were dealing with each output
        # dimension in batch
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


In [5]:
# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)
num_epochs = 2 if smoke_test else 2000

random_seeds = [42,43,46]
v_pred_mean_list_l1 = []
v_pred_std_list_l1 = []
te_pred_mean_list_l1 = []
te_pred_std_list_l1 = []
t_pred_mean_list_l1 = []
saved_model_list_l1 = []


for i in range(3):
    torch.manual_seed(random_seeds[i])
    np.random.seed(random_seeds[i])
    random.seed(random_seeds[i])
    
    model = MultitaskGPModel_l1()
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=num_tasks_l1)

    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam([
        {'params': model.parameters()},
        {'params': likelihood.parameters()},
    ], lr=0.05)

    # Our loss object. We're using the VariationalELBO, which essentially just computes the ELBO
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y_l.size(0))

    # We use more CG iterations here because the preconditioner introduced in the NeurIPS paper seems to be less
    # effective for VI.
    epochs_iter = tqdm.tqdm_notebook(range(num_epochs), desc="Epoch")

    min_val_loss = float('inf')
    patience = 100
    wait = 0

    for j in epochs_iter:
        # Within each iteration, we will go over each minibatch of data
        model.train()
        likelihood.train()

        optimizer.zero_grad()
        output = model(train_x_l)
        loss = -mll(output, train_y_l)

        epochs_iter.set_postfix(loss=loss.item())
        loss.backward()
        optimizer.step()
        
        
        # Make predictions
        model.eval()
        likelihood.eval()
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            val_predictions = likelihood(model(val_x_l))
            val_mean = val_predictions.mean.detach().numpy()
            val_loss = np.mean(np.abs(val_y_l.numpy() - val_mean))
            if val_loss < min_val_loss:
                wait = 0
                min_val_loss = val_loss
                saved_model = model
            elif val_loss >= min_val_loss:
                wait += 1
                if wait == patience:
#                     saved_model_list_l1.append(saved_model)
#                     te_predictions = likelihood(saved_model(test_x_l))
#                     te_mean = te_predictions.mean
#                     te_pred_mean_list_l1.append(te_mean)
#                     te_pred_lower, te_pred_upper = te_predictions.confidence_region()
#                     te_pred_std = te_mean - te_pred_lower
#                     te_pred_std_list_l1.append(te_pred_std)
                    
#                     v_predictions = likelihood(saved_model(val_x_l))
#                     v_mean = v_predictions.mean
#                     v_pred_mean_list_l1.append(v_mean)
#                     v_pred_lower, v_pred_upper = v_predictions.confidence_region()
#                     v_pred_std = v_mean - v_pred_lower
#                     v_pred_std_list_l1.append(v_pred_std)
                    
#                     t_predictions = likelihood(saved_model(train_x_l))
#                     t_mean = t_predictions.mean
#                     t_pred_mean_list_l1.append(t_mean)
                    break
        
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        saved_model_list_l1.append(saved_model)
        te_predictions = likelihood(saved_model(test_x_l))
        te_mean = te_predictions.mean
        te_pred_mean_list_l1.append(te_mean)
        te_pred_lower, te_pred_upper = te_predictions.confidence_region()
        te_pred_std = te_mean - te_pred_lower
        te_pred_std_list_l1.append(te_pred_std)

        v_predictions = likelihood(saved_model(val_x_l))
        v_mean = v_predictions.mean
        v_pred_mean_list_l1.append(v_mean)
        v_pred_lower, v_pred_upper = v_predictions.confidence_region()
        v_pred_std = v_mean - v_pred_lower
        v_pred_std_list_l1.append(v_pred_std)

        t_predictions = likelihood(saved_model(train_x_l))
        t_mean = t_predictions.mean
        t_pred_mean_list_l1.append(t_mean)

/var/folders/2g/wdp07rt57lg1cc7zgtgnr3yh0000gn/T/ipykernel_11508/2199203907.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  epochs_iter = tqdm.tqdm_notebook(range(num_epochs), desc="Epoch")


Epoch:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2000 [00:00<?, ?it/s]

In [7]:
# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)
num_epochs = 2 if smoke_test else 2000

random_seeds = [42,43,46]
pred_mean_list_l2 = []
pred_std_list_l2 = []
saved_model_list_l2 = []


for i in range(3):
    torch.manual_seed(random_seeds[i])
    np.random.seed(random_seeds[i])
    random.seed(random_seeds[i])
    
    model = MultitaskGPModel_l2()
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=num_tasks_l2)

    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam([
        {'params': model.parameters()},
        {'params': likelihood.parameters()},
    ], lr=0.05)

    # Our loss object. We're using the VariationalELBO, which essentially just computes the ELBO
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y_h.size(0))

    # We use more CG iterations here because the preconditioner introduced in the NeurIPS paper seems to be less
    # effective for VI.
    epochs_iter = tqdm.tqdm_notebook(range(num_epochs), desc="Epoch")

    min_val_loss = float('inf')
    patience = 100
    wait = 0

    for j in epochs_iter:
        # Within each iteration, we will go over each minibatch of data
        model.train()
        likelihood.train()

        optimizer.zero_grad()
        train_xx = torch.cat((t_pred_mean_list_l1[i][:train_x_h.shape[0],:], train_x_h),-1)
        output = model(train_xx)
        loss = -mll(output, train_y_h)

        epochs_iter.set_postfix(loss=loss.item())
        loss.backward()
        optimizer.step()
        
        
        # Make predictions
        model.eval()
        likelihood.eval()
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            val_xx = torch.cat((v_pred_mean_list_l1[i][:val_x_h.shape[0],:], val_x_h),-1)
            val_predictions = likelihood(model(val_xx))
            val_mean = val_predictions.mean.detach().numpy()
            val_loss = np.mean(np.abs(val_y_h.numpy() - val_mean))
            if val_loss < min_val_loss:
                wait = 0
                min_val_loss = val_loss
                saved_model = model
            elif val_loss >= min_val_loss:
                wait += 1
                if wait == patience:
#                     saved_model_list_l2.append(saved_model)
#                     test_xx = torch.cat((t_pred_mean_list_l1[i][:test_x_h.shape[0],:], test_x_h),-1)
#                     predictions = likelihood(saved_model(test_xx))
#                     mean = predictions.mean
#                     pred_mean_list_l2.append(mean)
                    
#                     pred_lower, pred_upper = predictions.confidence_region()
#                     pred_std = mean - pred_lower
#                     pred_std_list_l2.append(pred_std)
                    break
        
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        saved_model_list_l2.append(saved_model)
        test_xx = torch.cat((t_pred_mean_list_l1[i][:test_x_h.shape[0],:], test_x_h),-1)
        predictions = likelihood(saved_model(test_xx))
        mean = predictions.mean
        pred_mean_list_l2.append(mean)
        
        pred_lower, pred_upper = predictions.confidence_region()
        pred_std = mean - pred_lower
        pred_std_list_l2.append(pred_std)
        

/var/folders/2g/wdp07rt57lg1cc7zgtgnr3yh0000gn/T/ipykernel_11508/4034186289.py:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  epochs_iter = tqdm.tqdm_notebook(range(num_epochs), desc="Epoch")


Epoch:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2000 [00:00<?, ?it/s]

In [9]:
def nll_metric(pred_mu, pred_std, y):
#         pred_mu = torch.from_numpy(pred_mu)
#         pred_std = torch.from_numpy(pred_std)
#         y = torch.from_numpy(y)
        pred_mu = pred_mu.reshape(-1,6,87,87)
        pred_std = pred_std.reshape(-1,6,87,87)
        y = y.reshape(-1,6,87,87)
        nll_list = []
        for i in range(6):
            gaussian = torch.distributions.Normal(pred_mu[:,i], pred_std[:,i])
            nll = -gaussian.log_prob(y[:,i])
            nll = torch.mean(nll).detach().numpy().item()
            nll_list.append(nll)
        return nll_list

def mae_metric(y_pred, y_true):
    y_pred = y_pred.reshape(-1,6,87,87)
    y_true = y_true.reshape(-1,6,87,87)
    loss_list = []
    for i in range(6):
        loss = torch.abs(y_pred[:,i] - y_true[:,i])
        loss[loss != loss] = 0
        loss = loss.mean().detach().numpy().item()
        loss_list.append(loss)
    return loss_list

In [10]:
# MAE
# truth = test_y.reshape(-1,6,87,87)
truth = test_y_h
mae_list = []
nll_list = []
for i in range(3):
    mean = pred_mean_list_l2[i]
    std = pred_std_list_l2[i]
#     mean = mean.reshape(-1,6,87,87)
    mae = mae_metric(mean,truth)
    mae_list.append(mae)
    nll = nll_metric(mean,std,truth)
    nll_list.append(nll)
    
nll_arr = np.array(nll_list)
mae_arr = np.array(mae_list)

nll_mean = np.mean(nll_arr, 0)
nll_std = np.std(nll_arr, 0)
mae_mean = np.mean(mae_arr, 0)
mae_std = np.std(mae_arr, 0)

print(mae_mean,nll_mean)

[0.93210858 0.90160795 0.92718748 0.88728025 0.92905305 0.88155997] [2.30316194 2.30030958 2.30303033 2.29980357 2.30133001 2.29443741]


In [11]:
# MAE
# truth = test_y.reshape(-1,6,87,87)
truth = test_y_h
mae_list = []
nll_list = []
for i in range(3):
    mean = pred_mean_list_l2[i]
    std = pred_std_list_l2[i]
#     mean = mean.reshape(-1,6,87,87)
    mae = mae_metric(mean,truth)
    mae_list.append(mae)
    nll = nll_metric(mean,std,truth)
    nll_list.append(nll)
    
print(mae_list,nll_list)



[[0.9321095943450928, 0.9016100168228149, 0.9271917343139648, 0.8872880935668945, 0.9290631413459778, 0.8815695643424988], [0.9321101903915405, 0.9016080498695374, 0.9271921515464783, 0.8872836828231812, 0.9290578365325928, 0.8815655708312988], [0.9321059584617615, 0.9016057848930359, 0.9271785616874695, 0.8872689604759216, 0.9290381669998169, 0.8815447688102722]] [[2.306328058242798, 2.3038389682769775, 2.3065550327301025, 2.303262710571289, 2.304896831512451, 2.2979745864868164], [2.3079919815063477, 2.3047077655792236, 2.3073601722717285, 2.304313898086548, 2.305748701095581, 2.2991678714752197], [2.295165777206421, 2.292382001876831, 2.295175790786743, 2.2918341159820557, 2.293344497680664, 2.2861697673797607]]


In [15]:
np.save('result/nargp_mae.npy', np.array(mae_list))
np.save('result/nargp_nll.npy', np.array(nll_list))
np.save('result/nargp_mean.npy', torch.stack(pred_mean_list_l2,0).detach().numpy())
np.save('result/nargp_std.npy', torch.stack(pred_std_list_l2,0).detach().numpy())
np.save('result/nargp_truth.npy', test_y_h.detach().numpy())